# Part 1

In [1]:
import numpy as np

In [2]:
grid = np.genfromtxt("inputs/day14.txt", delimiter=1, dtype=str, comments=None)

In [3]:
tilted = list()
for col in grid.T:
    parts = np.split(col, np.unique(np.concatenate([np.argwhere(col=='#')[:,0], np.argwhere(col=='#')[:,0]+1])))
    tilted.append(np.concatenate([np.sort(p)[::-1] for p in parts]))

np.dot(np.arange(len(grid), 0, -1), (np.array(tilted).T == 'O').sum(axis=1))

113525

# Part 2

In [4]:
from functools import cache

def tilt(grid, dir):
    n_90_spins = {
        'N': 0,
        'E': 1,
        'S': 2,
        'W': -1,
    }
    
    grid = np.rot90(grid, k=n_90_spins[dir])

    tilted = list()
    for col in grid.T:
        parts = np.split(col, np.unique(np.concatenate([np.argwhere(col=='#')[:,0], np.argwhere(col=='#')[:,0]+1])))
        tilted.append(np.concatenate([np.sort(p)[::-1] for p in parts]))

    return np.rot90(np.array(tilted).T, k=-n_90_spins[dir])


@cache
def cycle(grid_bytes, grid_shape):
    grid = np.frombuffer(grid_bytes, dtype=np.dtype('<U1')).reshape(grid_shape)
    
    for dir in ['N', 'W', 'S', 'E']:
        grid = tilt(grid, dir)

    return grid.tobytes()

In [5]:
n=1_000_000_000
mod_not_updated = mod = 0

grid_bytes, grid_shape = grid.tobytes(), grid.shape

for i in range(n):
    grid_bytes = cycle(grid_bytes, grid_shape)

    # Use the cache to figure out size of cycle 
    
    # Allow a bit of burn in
    if i == 1000:
        cycle.cache_clear()
        mod = 0

    if cycle.cache_info().misses > mod:
        mod = cycle.cache_info().misses
        mod_not_updated = 0
    else:
        mod_not_updated +=1
    if mod_not_updated >= 1000:
        if (i-n) % mod == (-1 % mod):
            break

grid_out = np.frombuffer(grid_bytes, dtype=np.dtype('<U1')).reshape(grid_shape)
np.dot(np.arange(len(grid), 0, -1), (grid_out == 'O').sum(axis=1))

101292